In [2]:
from data_lib.data_lib import *

In [3]:
some_letters = read_data('AOMBPZKCEF.csv')

In [4]:
letters_map = {'A':1,'O':2,'M':3,'B':4,'P':5,'Z':6,'K':7,'C':8,'E':13,'F':14}

In [5]:
gests = read_data('data2.csv')

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Reshape
from keras.layers import LSTM

Using Theano backend.


In [7]:
from sklearn.metrics import confusion_matrix
gests = add_noise(gests, 0.5, 100)

In [8]:
X, y = array_from_dict(gests, ndots=-1, gyro=False)

In [9]:
n_units=100
n_features=3
n_epoch=10
model = Sequential()  # Creates a model
model.add(LSTM(n_units, input_dim=n_features)) 
model.add(Dense(16, activation='sigmoid'))  # Kind of output layer
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [10]:
model.load_weights('gests_lstm_weights')

In [12]:
def letters_predicted_sequences(model, letters, win_size=15, proba_threshold=0.5):
    X = []
    letters_array, y = array_from_dict(letters, ndots=-1, gyro=False)
    for letter in letters_array:
        seq = []
        k = 0
        while len(letter[0])>k+win_size:
            p = model.predict_proba(np.array([letter[0][k:k+win_size]]), verbose=0)[0]
            p = p / np.sum(p)
            p = np.where(p > proba_threshold)[0]
            if p:
                seq.append(p)
            k += 1
        X.append(np.array([seq]))
    return X, y
        

In [13]:
#some_letters = add_noise(some_letters, 0.5, 100)

In [14]:
r,t = letters_predicted_sequences(model, some_letters)

In [15]:
t

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14])

In [19]:
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn.dummy import DummyClassifier
from xgboost.sklearn import XGBClassifier
from sklearn import grid_search
from sklearn.metrics import accuracy_score

In [34]:
X = []
for x in r:
    if len(x.flatten())<2:
        X.append([x.flatten()[0]]*10)
    else:
        X.append(inter([x.flatten()],ndots=10)[0])

In [39]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, t, test_size=0.2, random_state=1)

In [44]:
xgb = XGBClassifier(colsample_bytree=0.4, learning_rate=0.05, max_depth=10, n_estimators=200, subsample=0.6)
scores_xgb = cross_validation.cross_val_score(xgb, np.array(X), t, cv=4)
print np.mean(scores_xgb)

0.822619047619
